In [ ]:

import vendors2.kibot.data.load.futures_forward_contracts as vkdlfu
import vendors2.kibot.data.load.s3_data_loader as vkdls3
import vendors2.kibot.metadata.load.kibot_metadata as vkmlki

In [ ]:
lfc_hc = vkmlki.KibotHardcodedContractLifetimeComputer(365, 7)

lfc_hc.compute_lifetime("CLJ17")

In [ ]:
lfc_ta = vkmlki.KibotTradingActivityContractLifetimeComputer()

lfc_ta.compute_lifetime("CLJ17")

In [ ]:
##

In [ ]:
symbols = ["ES", "CL"]
file = "../contracts.csv"


fcl = vkmlki.FuturesContractLifetimes(file, lfc_hc)

In [ ]:
fcl.save(["CL"])

In [ ]:
cl_data = fcl.load(["CL"])

In [ ]:
cl_data["CL"].head()

In [ ]:
fcem = vkmlki.FuturesContractExpiryMapper(cl_data)

In [ ]:
fcem.get_nth_contract("CL", "2010-01-01", 1)

In [ ]:
srs = fcem.get_nth_contracts("CL", "2010-01-10", "2010-01-20", freq="B", n=1)

In [ ]:
srs

In [ ]:
kdl = vkdls3.S3KibotDataLoader()

In [ ]:
ffc_obj = vkdlfu.FuturesForwardContracts(kdl)

In [ ]:
ffc_obj._replace_contracts_with_data(srs)